In [ ]:
import scipy.misc as misc
import skimage.filters
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label
from skimage import measure
from skimage.filters import threshold_otsu
from scipy import ndimage
import math as m

def raw_moment(data, i_order, j_order):
    (nrows, ncols) = data.shape
    y_indices, x_indicies = np.mgrid[:nrows, :ncols]
    return (data * x_indicies**i_order * y_indices**j_order).sum()


def moments_cov(data):
    data_sum = data.sum()
    m10 = raw_moment(data, 1, 0)
    m01 = raw_moment(data, 0, 1)
    x_centroid = m10 / data_sum
    y_centroid = m01 / data_sum
    u11 = (raw_moment(data, 1, 1) - x_centroid * m01) / data_sum
    u20 = (raw_moment(data, 2, 0) - x_centroid * m10) / data_sum
    u02 = (raw_moment(data, 0, 2) - y_centroid * m01) / data_sum
    cov = np.array([[u20, u11], [u11, u02]])
    return cov

## Methode 1


def distance_centre_masse(Image,patch):
    img=misc.imread(Image, flatten=True)
    centre_masse=ndimage.measurements.center_of_mass(img)
    centres_patch=[]
    centre1=[]
    centre2=[]

    #Centre des patchs

    for i in range (0,len(patch)-1):
        for p in range (0,20): #20 patchs par image
            for xy in range (0,2) : 
                pa = patch[i][p][0][xy] + 10
                if xy == 0:
                    x = pa
                else :
                    y = pa
            centres_patch.append([x,y])
    distance=[]
    for i in range (len(centres_patch)):
        distance.append(m.sqrt((centres_patch[i][0]-int(centre_masse[0]))**2+(centres_patch[i][1]-int(centre_masse[1]))**2))

    return distance

## Methode 2

def interieur_exterieur(Image,patch):
        #Centre des patchs
    centres_patch=[]
    for i in range (0,len(patch)-1):
        for p in range (0,20): #20 patchs par image
            for xy in range (0,2) : 
                pa = patch[i][p][0][xy] + 10
                if xy == 0:
                    x = pa
                else :
                    y = pa
            centres_patch.append([x,y])        
    img = misc.imread(Image, flatten=True)
    cov = moments_cov(img) #Matrice des covariances
    evals, evecs = np.linalg.eig(cov) #Valeurs Propres, Vecteurs propres
    
    # definir si le point est dans l'ellipse ou non 

    ## vecteur 1
    a1 = evecs[0][1]-evecs[0][0]
    b1 = evecs[0][1]-a1

    ## vecteur 2
    a2 = evecs[1][1]-evecs[1][0]
    b2 = evecs[1][1]-a2
    
    # definir le centre de l'ellipse
    mat1 = np.array([[1,-a1],[1,-a2]])
    mat2 = np.array([b1,b2])
    centre_ellipse = np.linalg.solve(mat1,mat2)
    
    result=[]
    for i in range (len(centres_patch)):
        X= evecs[0][1]-evecs[0][0] #longeur
        Y=evecs[1][1]-evecs[1][0] #largeur
        val = ( (centres_patch[i][0] - centre_ellipse[0])/X )**2 + ( (centres_patch[i][1] - centre_ellipse[1])/Y )**2
        if val < 1 :
            result.append(1)  # le patch est dans l'ellipse
        else :
            result.append(0)
    return result